# Boost Beamtime Productivity - NB 1
with databroker (V1) and supplemental pandas


### Goals
* Learn the 3 key features of bluesky API to maximize productive with reduced effort
* Use scalar values to track the "pulse" of your experiment that requires heavy image processing
* Exploit databroker V1
    * what did my collaborator do while i was asleep
    * avoid transcription errors and focus on the science
    * reconstruct inherited projects that are poorly documented
* Supplement with pandas to reduce "book-keeping" errors
* Supplement with pandas to make all data easier to interpret


# The most under-utilized feature in databroker
## the start document search </br>  


<details>
<summary><em>Getting data by scan_id too much trouble - I want to spend my time on science, not book-keeping</em></summary>
    <p> 
            

```python
db??
```

```python
Call docstring:
Search runs based on metadata.

This function returns an iterable of :class:`Header` objects. Each
Header encapsulates the metadata for a run -- start time, instruments
used, and so on, and provides methods for loading the data. In
addition to the Parameters below, advanced users can specify arbitrary
queries using the MongoDB query syntax.

The ``since`` and ``until`` parameters accepts the following
representations of time:

* timestamps like ``time.time()`` and ``datetime.datetime.now()``
* ``'2015'``
* ``'2015-01'``
* ``'2015-01-30'``
* ``'2015-03-30 03:00:00'``
```
</p>
</details> 


```python
hhs = db(since='2021-05-19 13:00:00', plan_name = 'count', purpose ='thermalize') 
```


### In the field example on a beamline that I have not used before (5+ years at NSLS-II)
**During Setup**: <ins>simple and custom report</ins> based on standard bluesky features
<details>
<summary><em>Run-report</em></summary>
    <p> 
        <img src="source_material/Boost_1_run_report.png">
</p>
</details>    </br>

**Day 1**: <ins>simple and custom analysis</ins> based on standard bluesky and with custom analysis cells for `db()`search
<details>
<summary><em>Custom analysis using pure top level databroker fuctions</em></summary> 
    <p> 
        <img src="source_material/Boost_1_day_1.png",width="500">
</p>
</details>  

**Day 3**: <ins>found a problem</ins> as it happened, not 3 hours later

<details>
<summary><em>Custom analysis using pure top level databroker fuctions</em></summary>
    <p> 
        <img src="source_material/Boost_1_day_3.png">
</p>
</details>  

## The three most important things in the bluesky API that made this possible

* **bluesky custom metadata** per scan (via bluesky's pre-assembled and stub plans)
    - https://blueskyproject.io/bluesky/plans.html

    ```python
       def my_plan():
          for T in temperatures:
                yield from mvr(theta, -5)
                yield from count([det], num=25, md = {'purpose':'background'})
                yield from mvr(theta, 5)
                yield from count([det], num=25, md = {'purpose':'signal'})
                yield from mv(cryostat_temperature, T)
                
        RE(my_plan())
    ```

* **bluesky baseline documents** - values of beamline objects **before** and **after** *each* scan
    - https://blueskyproject.io/tutorials/Supplemental%20Data.html#Baseline-Readings
    - setup bsui
    ```python
    sd.baseline = [diffractometer, cryostat_temperature, energy .....]
    ```
    - in analysis
    ```python
    db[scan_number].table('baseline')
    ```
* **ophyd EpicsSignals** - customized for your experiment
    - https://blueskyproject.io/tutorials/Epics%20Signal.html
    - setup bsui (or in a user script (`%run -i my_experiment_details.py`)
    ```python
        from ophyd.signal import EpicsSignalRO
    
        cryostat_sample_temperature = EpicsSignalRO('XF:00ID{ES-TCntl:1}Temp:B-RB',
                                                   name = 'cryostat_sample_temperature')
        sd.baseline.append([cryostat_sample_temperature])
    ```

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import cm
from itertools import cycle

import pandas as pd 
import numpy as np

%run -i run_report.py

In [ ]:
from download_example_data import fetch_example
db = fetch_example()

## Scenario: Quick investigation of potentially interesting sample
* What we know do we know prior to measurements?
    * Co L3 edge measurements
    * Measurements occurred on **2021-03-20 between 1pm and 11:59pm**
* Assumption: measurements looking for interesting things
* Critical points for beamline ophyd objects:

```
    tardis            :: name of diffractometer motors 
    tardis.delta      :: diffractometer 2-theta motor
    fccd              :: main area detector, EPICS AreaDectector ROIs to capture statistics
    fccd.stats1.total :: acts as detector slit for `tardis.delta`
    fccd.stats4.total :: acts as a TFY detector when fccd is not intercepting a Bragg condition 
    stemp.temp.A.T    :: cryostat temperature (K)
    stemp.temp.B.T    :: sample temperature  (K)
    pgm.energy        :: incident beamline energy (eV)
    epu2.phase        :: undulator phase for source polarization 
                            - epu2_phase_readback is BEST
                            - LH ( linear horizontal =  0.0 )
                            - LV ( linear vertical)  = 24.6 )
    sx                :: horizontal positioning of sample (~100um beam with much larger sample)
                            - cryostat's thermal expansion is in this direction
                            - "groups" of sx are likely the "same" spot for the "same" temperature
    
    NOTE: fccd.stats are in units of ADU with a dark image applied by the EPICS AD Stats Plugin. 
          EPICS AD Stats reports integrated intensity for 1 frame.
```



In [ ]:
my_data = print_scans(hhs = db(since = '2021-03-20 14:50:00', until = '2021-03-20 23:50:00', motors='pgm_energy', detectors = 'fccd'))

### XAS scans during initial exploration of sample

In [ ]:
hhs = db(since = '2021-03-20 14:50:00', until = '2021-03-20 23:50:00', motors='pgm_energy', detectors = 'fccd')
roi = str(4)

fig, ax1 = plt.subplots(figsize=(5,5))
for h in hhs:
    ## COLLECT DATA
    purpose = h.start.get('purpose','?')
    if purpose[0:3]=='xas':
        scan = h.start['scan_id']
        t = h.table()
        tb = h.table("baseline")
        sx = tb['sx'].mean()
        Tsample = tb['stemp_temp_B_T'].mean()
        phase = round(abs(tb['epu2_phase_readback'].mean()),0)
        config_attrs = h.descriptors[0]["configuration"]["fccd"]["data"]
        roi_x = config_attrs["fccd_roi" + roi + "_size_x"]
        roi_y = config_attrs["fccd_roi" + roi + "_size_y"]
        if phase < 20: phase ='H'
        else: phase = 'V'

        x = t.pgm_energy_setpoint
        y = t['fccd_stats'+roi+'_total']

        ## PLOTTING CONTROL
        ax1.plot(x, y, label = f'{Tsample:.0f}K roi{roi} {roi_x:>3}x{roi_y:>3} pix$^2$ L{phase} sx={sx:.2f} S{scan}')

ax1.legend(bbox_to_anchor=(1.1, 1.05))

### Issue 1:
* **science 101: graphs have labels**
<details>
<summary><em>Add generic axis labels by relying on `pandas`</em></summary>
    <p> 
            
    ```python
    ax1.set(xlabel=x.name,ylabel=y.name)
    ```

    **CHALLENGE** exercise for the reader: discover `hints` in `header.descriptors[0]`<br>  
        * this is contengent of beamline's bluesky profile configuration
        * ongoing work to improve usability of recorded units and labels
</p>
</details> 

In [ ]:
## WORKSPACE 


### Issue 2:
* **What general things should I turn into functions?** 
<details>
<summary><em>Hint: what is defined outside of flow control?</em></summary>
<p>
    
    ```python
        config_attrs = h.descriptors[0]["configuration"]["fccd"]["data"]
        roi_x = config_attrs["fccd_roi" + roi + "_size_x"]
        roi_y = config_attrs["fccd_roi" + roi + "_size_y"] 
    ```
    Instead
    
    ```python
       %run -i fccd_image_functions.py
       
       roi_x, _, roi_y, _, _ = get_fccd_roi(h, 'roi') 
    ```
    
</p>
</details> 

In [ ]:
## WORKSPACE 

### Issue 3:
* **I am tired and cannot invest effort to understand this plot or transcribe these numbers** 
<details>
<summary><em>or save time by making plots ready for the next proposal - what else can pandas do?</em></summary>
    <p>
        
    ```python
        
    hhs = db(since = '2021-03-20 14:50:00', until = '2021-03-20 23:50:00', 
                 motors='pgm_energy', detectors = 'fccd')
    roi = str(4)
    escan_dict = {'temperature': [], 'phase': [], 'sx':[], 'scan':[]}    #### *ANSWER 

    fig, ax1 = plt.subplots(figsize=(5,5))
    for h in hhs:
        ## COLLECT DATA
        purpose = h.start.get('purpose','?')
        if purpose[0:3]=='xas':
            scan = h.start['scan_id']
            t = h.table()
            tb = h.table("baseline")
            sx = tb['sx'].mean()
            Tsample = tb['stemp_temp_B_T'].mean()
            phase = round(abs(tb['epu2_phase_readback'].mean()),0)
            _, roi_x, _, roi_y, _  = get_fccd_roi(h, roi)                          #### *ANSWER
            if phase < 20: phase ='H'
            else: phase = 'V'

            x = t.pgm_energy_setpoint
            y = t['fccd_stats'+roi+'_total']

            ## PLOTTING CONTROL
            ax1.plot(x, y, label = f'{Tsample:.0f}K roi{roi} {roi_x:>3}x{roi_y:>3} pix$^2$ L{phase} sx={sx:.2f} S{scan}')
            for k,v in zip(escan_dict.keys(), [round(Tsample,0), phase, sx, scan]): #### *ANSWER 
                escan_dict[k].append(v)                                             #### *ANSWER 
    ax1.legend(bbox_to_anchor=(1.1, 1.05))
    ax1.set(xlabel=x.name,ylabel=y.name) #### *ANSWER 

    df_xas = pd.DataFrame(escan_dict)    #### *ANSWER 
    ```
</p>
</details> 

* https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html

In [ ]:
## WORKSPACE -- suggest to copy orignal from above and work on it here

### How can we understand the data without transcribing or couting the table like an imaginary abicus?
* https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_index.html 
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.count.html


<details>
<summary> **SUGGESTED SOLUTIONS** - <em>one line per cell</em></summary>
    <p> 
            
    ```python
    df_xas.set_index(["temperature",'sx','phase'])
    
    df_xas.set_index(["temperature",'sx','phase']).count(level='temperature')
    ```
</p>
</details> 

### How to  plot in meaningful ways without having to transcribe a single scan number (or `uid`)
* group data https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html
* if you can count members, then
    * <details>
<summary> **SUGGESTED SOLUTIONS** - <em>ALL in ONE cell</em></summary>
    <p> 
            
    ```python
    groups = df_xas.groupby(["sx","phase"])
    groups.count()
    ```
</p>
</details> 
* you can extract `scan_id` (named by `escan_dict["scan"]` or `header.start['scan_id']`)
    * <details>
<summary> **SUGGESTED SOLUTIONS** - <em>ALL in ONE cell</em></summary>
    <p> 
            
    ```python
    for group, frame in groups:                  
        print(group, frame["scan"].tolist())      
    ```
</p>
</details> 

### Let's get plotting
* if we use the flow control appoach above, things can get messy
* first let's convert all that to a **list** of **tuples** consisting of a **list and tuple**
    * <details>
<summary> **SUGGESTED SOLUTIONS** - <em>ALL in ONE cell</em></summary>
    <p> 
            
    ```python
        group_scans = [(frame["scan"].tolist(), group) for group, frame in df_xas.groupby(["sx","phase"])]
    ```
</p>
</details> 
* then we will make an customized color and line style to group the data
    * <details>
<summary> **SUGGESTED SOLUTIONS** - <em>ALL in ONE cell</em></summary>
    <p> 
            
    ```python
    roi = '4'

    colors = cycle(cm.get_cmap('viridis')(np.linspace(0, 1, len(group_scans))))

    fig, ax = plt.subplots(figsize=(5,5))
    for scans, group in group_scans:
        sx, pol = group
        hhs = db[scans]
        _, roi_x, _, roi_y, _  = get_fccd_roi(h, roi)
        color=next(colors)
        lines = cycle(['-','-.','--'])
        markers = cycle([None,])#'o','*','.'])
        for h in hhs:
            t=h.table() 
            x = t.pgm_energy_setpoint
            y = t['fccd_stats'+ roi +'_total']
            ax.plot(x, y, color=color, linestyle = next(lines), marker=next(markers),
                     label=f'{h["start"]["scan_id"]},  {sx:.3f}mm  L{pol}')
    ax.legend(bbox_to_anchor=(1.1, 1.05))                
    ax.set(xlabel=x.name,ylabel=f'{y.name} = ')
    ax.grid(True)
    ```
    
</p>
</details> 

In [ ]:
roi = '4'

colors = cycle(cm.get_cmap('viridis')(np.linspace(0, 1, len(group_scans))))


fig, ax = plt.subplots(figsize=(5,5))
for scans, group in group_scans:
    sx, pol = group
    hhs = db[scans]
    _, roi_x, _, roi_y, _  = get_fccd_roi(h, roi)
    color=next(colors)
    lines = cycle(['-','-.','--'])
    for h in hhs:
        t=h.table() 
        x = t.pgm_energy_setpoint
        y = t['fccd_stats'+ roi +'_total']
        ax.plot(x, y, color=color, linestyle = next(lines),
                 label=f'{h["start"]["scan_id"]},  {sx:.3f}mm  L{pol}')
ax.legend(bbox_to_anchor=(1.1, 1.05))                
ax.set(xlabel=x.name,ylabel=f'{y.name} = ')
ax.grid(True)

### Want to customize further
* Pick a different colormap
https://matplotlib.org/stable/gallery/color/colormap_reference.html<br> 
**SEMI-amerature TIP**: valid maps found in Exception of `cm.get_cmap('')`
* Extract roi size again and include in y-axis or legend
* Constrain limits
* Try to compare with detector slit roi (`roi = '1'`)
* Plot specific sx positions with 3 spectra
* Normalize spectra with assumption all have same off-resonance response


In [ ]:
## WORKSPACE for above


### Want to try reduce further?
* Make subplot for each unique condition: **"same"** position, temperature, and polarization
* Eliminate "bad" scans
* Reduce data dimentionality further by averaging


In [ ]:
# WORKSPACE for above

### But why do this 
<details>
<summary><em>instead of just keeping track of a few scan numbers?</em></summary>
    <p> 
Humans try to collect data efficiently when the parameter space is large!
<img src="source_material/shark.png", ,width="500">
        

</p>
</details> 


# Take aways
* Learning a little `pandas` is a good thing
* Investing time in generalized functions / notebooks makes 
    * future experiments easier
    * advanced data reduction and analysis more efficient 
* This workflow concept well to any beamline using `bluesky`
* `jupyter` can be a full analysis and book-keeping platform 
    - generate "reports" to host on "the cloud" 
    - keep track of bad scans, notes on data or analysis

### What if I get stuck
* easy ways to investigate document data per scan
* 4 main parts: start, stop, descriptors, stream (aka `.table()`)
```python
db[-1].start                   # shows rendering of the start document
list(db[-1].start)             # easy way to quickly investigate available keys
db[-1].table()                 # shows rendering of primary data stream - the most important
list(db[-1].table('baseline')) # easy way to quickly investigate available keys
db[-1].stream_names            # easy way to quickly investigate available argument values for .table()
```

### CHALLENGE:
* Investigate these and plot in a meaningful way.  
* Note the code block is equivalent to the very first "data collection and plotting" cell
    - ADDED
        - ```deltas = round(tb['tardis_delta'].mean())```
    - REMOVE
        - dictionary and pandas DataFrame creation
        - ```if purpose[0:3]=='xas': ```
* Try to target a specific purpose
    - This can be done with any keys in the start document

In [ ]:
hhs = db(since = '2021-03-20 14:50:00', until = '2021-03-20 23:50:00', motors='pgm_energy', detectors = 'fccd')

roi = str(1)
scans = []

fig, axes = plt.subplots(figsize=(5,5))
for h in hhs:
    purpose = h.start.get('purpose','?')
    scan = h.start['scan_id']
    t = h.table()
    tb = h.table("baseline")
    sx = round(tb['sx'].mean(),3)
    deltas = round(tb['tardis_delta'].mean())             #### ADDED THIS LINE COMPARED TO OUR INTIALS CELL
    Tsample = tb['stemp_temp_B_T'].mean()   
    phase = round(abs(tb['epu2_phase_readback'].mean()),0)
    _, roi_x, _, roi_y, _ = get_fccd_roi(h, roi)
    if phase < 20: phase ='H'
    else: phase = 'V'; 
    x = t.pgm_energy_setpoint
    y = t['fccd_stats'+roi+'_total']
    axes.plot(x, y, label = f'{Tsample:.0f}K roi{roi} {roi_x:>3}x{roi_y:>3} pix$^2$ {phase} {sx:.2f}mm S{scan}')

    scans.append(h.start['scan_id'])

axes.legend(bbox_to_anchor=(1.1, 1.05))

In [ ]:
%matplotlib widget

In [ ]:
%matplotlib inline